# Определение возраста покупателей

**Цель работы** — построить модель, определяющую возраст человека по изображению. Она будет работать в системе компьютерного зрения для фотофиксации покупателей супермаркетов в прикассовой зоне. Что в свою очередь поможет в дальнейшем предлагать релевантные товары покупателям из конкретных возрастных групп, а также дополнительно контролировать продажу товаров с возрастными ограничениями. 

Итоговая ошибка модели MAE по требованию заказчика должна быть не более 8 лет.

Для обучения используется набор фото людей с разметкой их возраста с сайта [ChaLearn Looking at People](https://chalearnlap.cvc.uab.cat/dataset/26/description/).

**Ход работы**

После загрузки данных проведём их EDA, подготовим к обучению.

Обучим нейронную сеть, оценим её качество на тестовой выборке.

Сформулируем общий вывод.
 
Таким образом работа будет состоять из следующих этапов:

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных-и-исследовательский-анализ" data-toc-modified-id="Загрузка-данных-и-исследовательский-анализ-1">Загрузка данных и исследовательский анализ</a></span></li><li><span><a href="#Построение-и-обучение-модели" data-toc-modified-id="Построение-и-обучение-модели-2">Построение и обучение модели</a></span></li><li><span><a href="#Оценка-качества-модели" data-toc-modified-id="Оценка-качества-модели-3">Оценка качества модели</a></span></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-4">Итоговый вывод</a></span></li></ul></div>

## Загрузка данных и исследовательский анализ

In [1]:
import os
import random
from itertools import product

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
SEED = 3
set_seed(SEED)

Проведём небольшой EDA, в том числе оценив распределение возраста и взглянув на примеры фото из датасета.

In [3]:
labels = pd.read_csv('labels.csv')
display(
    labels.head(10),
    labels.sample(10, random_state=SEED),
    labels.info()
    )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7591 entries, 0 to 7590
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  7591 non-null   object
 1   real_age   7591 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 118.7+ KB


,file_name,real_age
0,000000.jpg,4
1,000001.jpg,18
2,000002.jpg,80
3,000003.jpg,50
4,000004.jpg,17
5,000005.jpg,27
6,000006.jpg,24
7,000007.jpg,43
8,000008.jpg,26
9,000009.jpg,39


,file_name,real_age
41,000041.jpg,34
3689,003689.jpg,20
6584,006589.jpg,16
2084,002084.jpg,65
994,000994.jpg,40
808,000808.jpg,27
4411,004411.jpg,66
6192,006195.jpg,31
4317,004317.jpg,34
4118,004118.jpg,18


None

Проверим, имеются ли пропуски или дубликаты.

In [4]:
print('Количество пропусков датасета —', labels.isna().sum().sum())
print('Количество дубликатов датасета —', labels.duplicated().sum())

Количество пропусков датасета — 0
Количество дубликатов датасета — 0


In [5]:
fig=go.Figure()

bin_size = 1
bin_start = 0.5
bin_end = 100

y_values, bin_edges = np.histogram(
    labels.real_age,
    bins=np.arange(bin_start, bin_end + bin_size, bin_size)
    )
x_values = (bin_edges[:-1] + bin_edges[1:]) / 2
coefficients = np.polyfit(x_values, y_values, 8)

x_line = np.linspace(x_values[0], x_values[-1], 100)
y_line = np.polyval(coefficients, x_line)

fig.add_trace(go.Histogram(
    x=labels.real_age,
    xbins=dict(start=bin_start, end=bin_end, size=bin_size)
    ))
fig.add_trace(go.Scatter(x=x_line, y=y_line))

fig.update_layout(
    width=1200,
    height=800,
    showlegend=False,
    xaxis_title='Возраст, лет',
    yaxis_title='Количество фото',
    title=dict(
        text='Распределение возраста в датасете',
        x=.5
        ))
fig.show()
print('Описательная статистика распределения возраста:')
labels.real_age.describe()

Описательная статистика распределения возраста:


count    7591.000000
mean       31.201159
std        17.145060
min         1.000000
25%        20.000000
50%        29.000000
75%        41.000000
max       100.000000
Name: real_age, dtype: float64

Посмотрим на примеры изображений в датасете.

In [6]:
datagen = ImageDataGenerator()
flow = datagen.flow_from_dataframe(
        dataframe=labels,
        directory='..\\projects\\determinig_the_age_of_buyers\\data',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        seed=SEED)
features, target = next(flow)

fig = make_subplots(
    4,
    4,
    subplot_titles=[str(t) for t in target],
    horizontal_spacing=.03,
    vertical_spacing=.03
    )
for i, j in product(range(4), range(4)):
    k = i * 4 + j
    fig.add_trace(
        px.imshow(features[k]).data[0],
        i+1,
        j+1
        ).update_xaxes(showticklabels = False) \
        .update_yaxes(showticklabels = False)
fig.update_layout(width=1000, height=1000)
fig.show()

Found 7591 validated image filenames.


**Промежуточные выводы**

Распределение возраста людей на фото похоже на распределение Пуассона за исключением повышенного количества фото детей раннего возраста (до 8 лет). Фотографий детей 8-10 лет достаточно мало, толко после данного возраста их число начинает расти. Также сильно мало фото людей от 80 лет, от 90 же вообще букально единицы. Веротно, модели несколько сложнее будет точно предсказывать возраст на указанных выше интервалах, но это нельзя назвать критичным, так как вряд ли супермаркету будет важно точно опознать людей данных возрастных групп.

Можно заметить пики фотографий людей с "круглым" возрастом: 30, 40, 50 лет и т.д. Возможно, это сделано намеренно составителями датасета, что может помочь модели лучше разделять группы людей между данными возрастами.

Средний возраст в районе 30, минимальный с макимальным (1 и 100), как и само распределение в целом выглядят достаточно правдоподобно.

Указанный возраст людей на фото выглядит также реалистично, видно, что сами изображения разного качества, однако достаточного для определения возраста. К фотографиям были изначально применены некоторые аугментации, что поможет использовать меньшее их количество во время использования загрузчика данных для обучения.

## Построение и обучение модели

Определим функции для подгрузки батчей во время обучения, а также для формирования и тренировки модели. Будем использовать архитертуру ResNet50, чтобы не слишком сильно переобучить модель на наших не очень объёмных данных, с изменённой головой для классификации. Применим некоторые дополнительные аугментации к изображениям для ещё большего улучшения качества модели.

In [12]:
def load_train(path):
    labels=pd.read_csv(path+'labels.csv')
    train_datagen=ImageDataGenerator(
        validation_split=.25,
        horizontal_flip=True,
        vertical_flip=True,
        channel_shift_range=100
        )
    train_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'data',
        x_col='file_name',
        y_col='real_age',
        subset='training',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        seed=SEED)
    return train_flow
 
def load_test(path):
    labels=pd.read_csv(path+'labels.csv')
    test_datagen = ImageDataGenerator(validation_split=.25)
    test_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'data',
        x_col='file_name',
        y_col='real_age',
        subset='validation',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        seed=SEED)
    return test_flow
 
def create_model(input_shape=(224, 224, 3)):
    backbone = ResNet50(
        input_shape=input_shape,
        weights='imagenet', 
        include_top=False
        )
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu')) 
    optimizer = Adam(learning_rate=0.0001)
    model.compile(
        optimizer=optimizer,
        loss='mean_absolute_error', 
        metrics=['mae']
        )
    return model
 
def train_model(
    model,
    train_data,
    test_data,
    batch_size=16,
    epochs=10,
    steps_per_epoch=None,
    validation_steps=None
    ):
    model.fit(
        train_data, 
        validation_data=test_data,
        batch_size=batch_size, 
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        verbose=2
        )
    return model

Сгенерируем обучающие и тестовые данные, а также экземляр модели.

In [13]:
path = '..\\projects\\determinig_the_age_of_buyers\\'
train_data = load_train(path)
test_data = load_test(path)

model = create_model()

Found 5694 validated image filenames.
Found 1897 validated image filenames.


In [14]:
training = train_model(model, train_data, test_data)

Epoch 1/10
356/356 - 488s - loss: 11.7447 - mae: 11.7447 - val_loss: 10.8484 - val_mae: 10.8484 - 488s/epoch - 1s/step
Epoch 2/10
356/356 - 483s - loss: 9.1519 - mae: 9.1519 - val_loss: 10.1853 - val_mae: 10.1853 - 483s/epoch - 1s/step
Epoch 3/10
356/356 - 506s - loss: 8.3308 - mae: 8.3308 - val_loss: 7.9867 - val_mae: 7.9867 - 506s/epoch - 1s/step
Epoch 4/10
356/356 - 503s - loss: 7.8032 - mae: 7.8032 - val_loss: 7.7562 - val_mae: 7.7562 - 503s/epoch - 1s/step
Epoch 5/10
356/356 - 492s - loss: 7.5375 - mae: 7.5375 - val_loss: 12.0142 - val_mae: 12.0142 - 492s/epoch - 1s/step
Epoch 6/10
356/356 - 494s - loss: 7.1492 - mae: 7.1492 - val_loss: 7.9760 - val_mae: 7.9760 - 494s/epoch - 1s/step
Epoch 7/10
356/356 - 500s - loss: 6.8896 - mae: 6.8896 - val_loss: 7.5818 - val_mae: 7.5818 - 500s/epoch - 1s/step
Epoch 8/10
356/356 - 495s - loss: 6.6631 - mae: 6.6631 - val_loss: 6.8679 - val_mae: 6.8679 - 495s/epoch - 1s/step
Epoch 9/10
356/356 - 498s - loss: 6.3666 - mae: 6.3666 - val_loss: 8.549

## Оценка качества модели

Построим сравнительный график настоящих тестовых данных и соответствующих предсказаний, чтобы точнее изучить результаты работы получившейся модели и её качество.

In [15]:
gt = labels.iloc[test_data.index_array]['real_age'].values
preds = model.predict(test_data, batch_size=16).flatten()
ids = np.argsort(gt)

119/119 [==============================] - 40s 336ms/step


In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.arange(gt.size),
    y=preds[ids],                         # для удобства интерпритации
    name='Предсказания',                  # отобразим тестовые примеры
    line=dict(color='crimson', width=1.5) # по возрастанию возраста
    ))
fig.add_trace(go.Scatter(
    x=np.arange(gt.size),
    y=gt[ids],
    name='Настоящий возраст',
    line=dict(color='darkblue', width=1.5)
    ))
fig.update_layout(
    width=1200,
    height=600,
    legend_orientation='h',
    title=dict(
        text='Предсказанный возраст по сравнению с настоящим',
        x=.5
        ),
    xaxis_title='Тестовый пример',
    yaxis_title='Возраст, лет'
    )
fig.show()

## Итоговый вывод

Качество модели можно назвать хорошим, при требуемом минимальном значении ошибки в 8 лет модель ошибается на ~6.8. Однако не стоит рекомендовать её для проверки корректности продаж товаров с возрастными ограничениями, так как 6-7 лет — слишком большой разброс для данной задачи.

По графику выше видно, что чаще и сильнее модель ошибается на возрастных промежутках до 18 и после 40 лет. В остальных случаях качество модели можно назвать более стабильным. Также можно отметить, что периодически модель путает детский возраст со старческим в обоих соотношениях.

Для увличения качества можно попробовать добавить один или несколько линейных слоёв к "голове" сети, использовать больший размер батча и дополнительно поэксперементировать с аугментациями и оптимизаторами.